<a href="https://colab.research.google.com/github/16Komang/Big-Data-Tugas-7/blob/main/ml_recommender_scala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.system("rm -rf /content/ml-predictions.csv")
os.system("rm -rf /content/mymodel")

0

In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=25b73cb4be47e7570606e159ffd21849b502b874d2c99e3e74517fd1b55df8a6
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, isnull

In [5]:
spark = SparkSession.builder.getOrCreate()

# Define the Rating class
class Rating:
    def __init__(self, userId, movieId, rating, timestamp):
        self.userId = userId
        self.movieId = movieId
        self.rating = rating
        self.timestamp = timestamp

# Define the parseRating function
def parseRating(str):
    fields = str.split("::")
    assert len(fields) == 4
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]), int(fields[3]))

# Test the parseRating function
parseRating("1::1193::5::978300760")

# Load the raw data
raw = spark.sparkContext.textFile("/content/drive/MyDrive/Big data/ml-1m/ratings.dat")

# Check one record
raw.take(1)

['1::1193::5::978300760']

In [6]:
# Parse the ratings and create a DataFrame
ratings = raw.map(parseRating).toDF()

# Check the ratings DataFrame
ratings.show(5)

# Split the data into training and test sets
training, test = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

# Save the model
model.save("mymodel")

+-------+------+---------+------+
|movieId|rating|timestamp|userId|
+-------+------+---------+------+
|   1193|   5.0|978300760|     1|
|    661|   3.0|978302109|     1|
|    914|   3.0|978301968|     1|
|   3408|   4.0|978300275|     1|
|   2355|   5.0|978824291|     1|
+-------+------+---------+------+
only showing top 5 rows



In [7]:
# Generate predictions on the test set
predictions = model.transform(test)

# Calculate the root mean square error (RMSE)
squared_error = predictions.withColumn("error", (predictions["rating"] - predictions["prediction"])**2)

valid_squared_error = squared_error.filter(~isnull(squared_error["error"]) & ~isnan(squared_error["error"]))

rmse = valid_squared_error.select("error").agg({"error": "mean"}).collect()[0][0] ** 0.5
print("RMSE: ", rmse)

# Show the first 10 predictions
predictions.show(10)

# Save the predictions as a CSV file
predictions.select("userId", "movieId", "rating", "prediction").write.csv("ml-predictions.csv")

RMSE:  0.8986796063562732
+-------+------+----------+------+----------+
|movieId|rating| timestamp|userId|prediction|
+-------+------+----------+------+----------+
|      1|   2.0|1002083483|   482| 3.7928982|
|      1|   2.0| 976214774|   491|  3.668443|
|      1|   2.0| 975196744|  1150|  2.918355|
|      1|   2.0| 974863438|  1167| 3.9370117|
|      1|   2.0| 974760465|  1434| 2.0482757|
|      1|   2.0| 983746388|  1710|  4.198209|
|      1|   2.0| 975812971|  2195| 3.1201231|
|      1|   2.0| 974602476|  2221| 4.4824157|
|      1|   3.0| 972779019|  2807| 3.5447724|
|      1|   2.0| 972057552|  2888| 3.8574185|
+-------+------+----------+------+----------+
only showing top 10 rows

